In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from collections import Counter
import datetime as dt

In [2]:
client = bigquery.Client()
dataset_ref = client.dataset("thelook_ecommerce", project="bigquery-public-data")


## Conversion Rates Analysis

Scope Definition: 
- Primary Question: Which traffic source contribute the most to the business conversion rates?
- Identify any potential seasonal, trending patterns over time.

Constraint Identification:
- These data can't be used to infer or predict detailed users behavior or sales related metrics.

In [3]:
query = """
WITH ordered_events AS (
  SELECT
    session_id,
    event_type,
    traffic_source,
    created_at,
    ROW_NUMBER() OVER (PARTITION BY session_id ORDER BY created_at) AS event_order
  FROM
    `bigquery-public-data.thelook_ecommerce.events`
)
SELECT
  session_id,
  MIN(CASE WHEN event_type = 'home' THEN event_order END) AS home_order,
  MIN(CASE WHEN event_type = 'department' THEN event_order END) AS department_order,
  MIN(CASE WHEN event_type = 'product' THEN event_order END) AS product_order,
  MIN(CASE WHEN event_type = 'cart' THEN event_order END) AS cart_order,
  MIN(CASE WHEN event_type = 'purchase' THEN event_order END) AS purchase_order,
  ARRAY_AGG(event_type ORDER BY event_order) AS event_sequence,
  ARRAY_AGG(traffic_source ORDER BY event_order LIMIT 1)[OFFSET(0)] AS first_touch_source,
  MIN(created_at) AS timestamp
FROM
  ordered_events
GROUP BY
  session_id
"""

conversion_df = client.query(query).to_dataframe()
conversion_df.describe()

,home_order,department_order,product_order,cart_order,purchase_order
count,87662.0,430538.0,680863.0,430670.0,180863.0
mean,1.002977,1.213296,1.762247,2.907851,7.152618
std,0.054693,0.41118,0.668749,0.709448,2.783314
min,1.0,1.0,1.0,1.0,4.0
25%,1.0,1.0,1.0,2.0,5.0
50%,1.0,1.0,2.0,3.0,7.0
75%,1.0,1.0,2.0,3.0,7.0
max,3.0,4.0,4.0,5.0,13.0


Clean, reformat the timestamp variable

In [4]:
dat = conversion_df.copy()

dat['timestamp'] = pd.to_datetime(dat['timestamp']).dt.tz_localize(None)
dat['date'] = dat['timestamp'].dt.date
dat['week'] = dat['timestamp'].dt.to_period('W').dt.start_time
dat['month'] = dat['timestamp'].dt.to_period('M').dt.start_time
dat['quarter'] = dat['timestamp'].dt.to_period('Q').dt.start_time

dat.head()

,session_id,home_order,department_order,product_order,cart_order,purchase_order,event_sequence,first_touch_source,timestamp,date,week,month,quarter
0,0b91e857-de1d-4982-b676-1c65cb5276bb,<NA>,<NA>,2,1,<NA>,"[cart, product, cancel]",Email,2019-07-01 10:40:00,2019-07-01,2019-07-01,2019-07-01,2019-07-01
1,11bb037a-02a0-4719-9cf3-45d69d5e7763,<NA>,<NA>,2,1,<NA>,"[cart, product, cancel]",Email,2020-10-14 17:18:00,2020-10-14,2020-10-12,2020-10-01,2020-10-01
2,144e27ab-b418-42b0-a157-88c23c3dfedd,<NA>,<NA>,2,1,<NA>,"[cart, product, cancel]",Organic,2025-06-13 00:37:00,2025-06-13,2025-06-09,2025-06-01,2025-04-01
3,1a97568f-7b20-4fc3-bc50-17d7f5edb142,<NA>,<NA>,2,1,<NA>,"[cart, product, cancel]",Adwords,2022-02-09 13:05:00,2022-02-09,2022-02-07,2022-02-01,2022-01-01
4,1d664d14-b1b3-4df5-8ec7-1db634d2e312,<NA>,<NA>,2,1,<NA>,"[cart, product, cancel]",Email,2024-01-26 10:36:00,2024-01-26,2024-01-22,2024-01-01,2024-01-01


Visualize the data

In [5]:
daily_stats = dat.groupby('date').agg(
    total_sessions=('session_id', 'count'),
    purchases=('purchase_order', lambda x: x.notna().sum())
).reset_index()
daily_stats['conversion_rate'] = (daily_stats['purchases'] / daily_stats['total_sessions']) * 100


fig_daily = px.line(daily_stats, x='date', y='conversion_rate', 
                     title='Daily Conversion Rate')
fig_daily.show();


In [6]:
weekly_stats = dat.groupby('week').agg(
    total_sessions=('session_id', 'count'),
    purchases=('purchase_order', lambda x: x.notna().sum())
).reset_index()
weekly_stats['conversion_rate'] = (weekly_stats['purchases'] / weekly_stats['total_sessions']) * 100


fig_weekly = px.line(weekly_stats, x='week', y='conversion_rate', 
                     title='Weekly Conversion Rate')
fig_weekly.show();

In [7]:
monthly_stats = dat.groupby('month').agg(
    total_sessions=('session_id', 'count'),
    purchases=('purchase_order', lambda x: x.notna().sum())
).reset_index()
monthly_stats['conversion_rate'] = (monthly_stats['purchases'] / monthly_stats['total_sessions']) * 100


fig_monthly = px.line(monthly_stats, x='month', y='conversion_rate', 
                     title='Monthly Conversion Rate')
fig_monthly.show();

In [8]:
quarterly_stats = dat.groupby('quarter').agg(
    total_sessions=('session_id', 'count'),
    purchases=('purchase_order', lambda x: x.notna().sum())
).reset_index()

fig_session_purchase = go.Figure()
fig_session_purchase.add_trace(go.Scatter(x=quarterly_stats['quarter'], y=quarterly_stats['total_sessions'], mode='lines', name='Total Sessions'))
fig_session_purchase.add_trace(go.Scatter(x=quarterly_stats['quarter'], y=quarterly_stats['purchases'], mode='lines', name='Purchases'))
fig_session_purchase.update_layout(title='Sessions and Purchases Over Time', xaxis_title='Date', yaxis_title='Count')
fig_session_purchase.show()

In [9]:
journeys = dat[dat['date'] >= dt.date(2023, 1, 1)]['event_sequence']

desired_order = ['Start', 'home', 'department', 'product', 'cart', 'purchase', 'cancel']

transitions = Counter()
for journey in journeys:
    journey = list(journey)
    transitions[('Start', journey[0])] += 1
    for a, b in zip(journey[:-1], journey[1:]):
        transitions[(a, b)] += 1

# Calculate base counts for each source node
base_counts = Counter()
for (a, b), v in transitions.items():
    base_counts[a] += v

# Filter links by conversion rate threshold (e.g., 3%)
MIN_CONV_RATE = 1.0 
filtered_transitions = {}
for (a, b), v in transitions.items():
    conv_rate = v / base_counts[a] * 100
    if conv_rate >= MIN_CONV_RATE:
        filtered_transitions[(a, b)] = v

# Only keep nodes that are in the desired order and present in filtered_transitions
present_nodes = set()
for a, b in filtered_transitions:
    present_nodes.add(a)
    present_nodes.add(b)
unique_labels = [label for label in desired_order if label in present_nodes]

label_idx = {label: i for i, label in enumerate(unique_labels)}

# Prepare source, target, value, and custom_labels lists
source = []
target = []
value = []
custom_labels = []

for (a, b), v in filtered_transitions.items():
    if a in label_idx and b in label_idx:
        source.append(label_idx[a])
        target.append(label_idx[b])
        value.append(v)
        base = base_counts[a]
        if base > 0:
            rate = v / base * 100
            custom_labels.append(f"{a} → {b}<br>{v} ({rate:.1f}%)")
        else:
            custom_labels.append(f"{a} → {b}<br>{v}")

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=unique_labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value,
        label=custom_labels
    )
))

fig.update_layout(title_text="Session Journey Sankey Diagram (Links ≥ 3% Conversion Rate)", font_size=12)
fig.show()

From this Sankey Diagram, it is indicative that most users follow a typical path: home -> department -> product -> cart -> purchase, with some drop-off at each stage. Additionally, the largest drop-off points are prime targets for optimization for this business. Test and deploy new features might improve these conversion rates even further.

In [10]:
# Define your traffic sources
sources = ['Email', 'Adwords', 'Facebook', 'Youtube', 'Organic']

source_stats = {}

for src in sources:
    stats = dat[dat['first_touch_source'] == src].groupby('quarter').agg(
        total_sessions=('session_id', 'count'),
        purchases=('purchase_order', lambda x: x.notna().sum())
    ).reset_index()
    stats['conversion_rate'] = (stats['purchases'] / stats['total_sessions']) * 100
    source_stats[src] = stats

fig_quarterly = go.Figure()

for i, src in enumerate(sources):
    stats = source_stats[src]
    fig_quarterly.add_trace(go.Scatter(
        x=stats['quarter'],
        y=stats['conversion_rate'],
        mode='lines+markers',
        name=src
    ))

fig_quarterly.update_layout(
    title='Quarterly Conversion Rates Across Sources',
    xaxis_title='Quarter',
    yaxis_title='Conversion Rate (%)',
    legend_title='Traffic Source'
)
fig_quarterly.show()

These suspiciously high and trending-upward conversion rates strongly indicate that the synthetic dataset does not convey accurate enough real-world user behavior and that further analysis will most likely result in incomprehensible results. In addition, this chart of conversion rates across sources highlights the likely insignificant differences between traffic sources' contributions to the business. 

Extra visualizations:

In [11]:
source_stats = dat.groupby('first_touch_source').agg(
    total_sessions = ('session_id', 'count'),
    purchases = ('purchase_order', lambda x: x.notna().sum())
)
source_stats['conversion_rate'] = (source_stats['purchases'] / source_stats['total_sessions']) * 100
source_stats = source_stats.reset_index()

fig_bar = px.bar(
    source_stats,
    x='first_touch_source',
    y='conversion_rate',
    title='Conversion Rate by Traffic Source',
    labels={'conversion_rate': 'Conversion Rate (%)', 'first_touch_source': 'Traffic Source'}
)
fig_bar.show()

As mentioned above, the contribution hardly differs from sources to sources.

In [12]:
pivot = dat.pivot_table(
    index='first_touch_source',
    columns='month',
    values='session_id',
    aggfunc='count'
)

fig = px.imshow(
    pivot,
    labels=dict(x="Month", y="Traffic Source", color="Session Count"),
    title="Session Count Heatmap by Source"
)
fig.show()

There seems to be a small spike in session count every February for Adwords, Email, Facebook, and Youtube. Organic seems to harbor the same pattern yet its effect is almost unnoticeable. In June 2025, there is an extreme spike for Email, potentially signaling a successful marketing campaign for that type of traffic source.

In [13]:
source_counts = dat.groupby('first_touch_source').agg(
    sessions=('session_id', 'count'),
).reset_index()

fig = px.pie(
    source_counts,
    names='first_touch_source',
    values='sessions',
    title='Session Distribution by Traffic Source'
)
fig.show()

This chart specifically highlights the contributions from different traffic sources, with Email being the biggest contributor.